In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import numpy as np
from scipy.io import wavfile
import tensorflow as tf
import utilities as util

import matplotlib.pylab as plt 

import warnings
warnings.filterwarnings("ignore", category=wavfile.WavFileWarning)

In [2]:
%%time
nps_uniq, dfreqs, notes = util.precompute_spect()
smallest_nps = 44100/24 * 4/3 
smallest_nps = 2**int(np.log2(smallest_nps))
note_freq = util.frequencies('A0', 'D8')
sample_files, target_vals = util.gather_samples(util.note_names('C1', 'C2'), 5) # Guitar, Trombone, Piano
init = tf.global_variables_initializer()


CPU times: user 9.09 ms, sys: 1.1 ms, total: 10.2 ms
Wall time: 9.88 ms


In [3]:
%%time

all_specs = []
for sf in sample_files:
    s = util.preprocess(sf, nps_uniq, dfreqs, notes, smallest_nps, note_freq)
    all_specs.append(s)

CPU times: user 1h 9min 26s, sys: 7.13 s, total: 1h 9min 33s
Wall time: 1h 9min 45s


In [4]:
time_steps = 88
n_inputs=time_steps*len(notes)

In [5]:
X = tf.placeholder(tf.float16, shape=(None,n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [6]:
%%time

n_hidden1 = 7
n_hidden2 = 42
n_outputs = 3

with tf.name_scope("network"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu) # Play with activations
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu) # Play with activations
    net_results = tf.layers.dense(hidden2, n_outputs, name="outputs")

CPU times: user 30.8 ms, sys: 0 ns, total: 30.8 ms
Wall time: 30.6 ms


In [7]:
%%time

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=net_results)
    loss = tf.reduce_mean(xentropy, name="loss")

CPU times: user 2.16 ms, sys: 898 µs, total: 3.06 ms
Wall time: 2.88 ms


In [8]:
%%time
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

CPU times: user 348 ms, sys: 2.99 ms, total: 351 ms
Wall time: 351 ms


In [9]:
%%time
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(tf.cast(net_results, tf.float32), y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 2.94 ms, sys: 0 ns, total: 2.94 ms
Wall time: 2.69 ms


In [ ]:
%%time

spectrograms = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    spectrograms = sess.run(all_specs)

In [11]:
%%time
new_specs22 = []
for i in range(len(all_specs)):
    new_spectrograms = []
    end = len(np.trim_zeros(spectrograms[i].sum(1), 'b'))
    new_spectrograms = np.concatenate(spectrograms[i][:,:end]).ravel()
    print(new_spectrograms.shape)
    new_specs22.append(new_spectrograms)

IndexError: list index out of range

In [12]:
%%time
saver = tf.train.Saver()

n_epochs = 10
batch_size = 3
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
#         for iteration in range(len(new_specs22) // batch_size):
        X_batch = new_specs22
        Y_batch = target_vals
        sess.run(training_op, feed_dict={X: np.stack(X_batch),
                                         y: np.array(Y_batch)})
#         acc_train = accuracy.eval(feed_dict={X: X_batch, y: [0,1,2]})
        acc_val = accuracy.eval(feed_dict={X: X_batch, y: Y_batch})
        print(epoch, "Train accuracy:", 0, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./FirstNetwork.ckpt")

ValueError: need at least one array to stack

In [13]:
new_sample_files = []
new_target_vals = []
new_location = '../samples/'
new_instruments = ['Trombone', 'Guitar', 'Piano']
for i, new_name in enumerate(new_instruments):
    for new_note in (util.note_names('C1', 'F1')):
        for j in range(2, 3):
            new_file_name = new_location + new_name+"/"+new_name +"_"+new_note[:-1]+"("+str(j)+").wav"
            new_sample_files.append(new_file_name)
            new_target_vals.append(i)

In [14]:
%%time
# Preprocess all of the sample files
# Create lists of all of the frequencies, times and spectrograms
new_all_freqs = note_freq[:-1]
new_all_specs = []
new_all_names = []
for sf in new_sample_files:
    # TODO: TF_SPEC needs all of these constants. Is there a way that we can reduce the parameters?
    new_s = util.preprocess(sf, nps_uniq, dfreqs, notes, smallest_nps, note_freq)
    new_all_specs.append(s)

CPU times: user 5min 58s, sys: 554 ms, total: 5min 59s
Wall time: 6min


In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    spectrograms2 = sess.run(new_all_specs)

In [16]:
%%time
new_specs33 = []
for i in range(len(new_all_specs)):
    new_spectrograms2 = []
    end = len(np.trim_zeros(spectrograms2[i].sum(1), 'b'))
    new_spectrograms2 = np.concatenate(spectrograms2[i][:,:end]).ravel()
    new_specs33.append(new_spectrograms2)

CPU times: user 1.04 ms, sys: 0 ns, total: 1.04 ms
Wall time: 740 µs


In [17]:
with tf.Session() as sess:
    saver.restore(sess, "./FirstNetwork.ckpt")
    X_new_scaled = new_specs33
    Z = net_results.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./FirstNetwork.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Raw_Freqs_100 not found in checkpoint
	 [[node save/RestoreV2 (defined at <timed exec>:1)  = RestoreV2[dtypes=[DT_DOUBLE, DT_DOUBLE, DT_DOUBLE, DT_DOUBLE, DT_DOUBLE, ..., DT_HALF, DT_HALF, DT_HALF, DT_HALF, DT_HALF], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "/usr/lib64/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib64/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/refactored-robot/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/refactored-robot/l

In [ ]:
y_pred

In [ ]:
Z

In [ ]:
new_target_vals